<a href="https://colab.research.google.com/github/abhishekkm8088/Pyspark-Abhi/blob/main/Pyspark_Scala_Usecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Spark Environment Setup

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

import os, findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
findspark.init()


In [2]:
from google.colab import files
uploaded = files.upload()

Saving customers.csv to customers.csv
Saving employees.csv to employees.csv
Saving offices.csv to offices.csv
Saving orderdetails.csv to orderdetails.csv
Saving orders.csv to orders.csv
Saving payments.csv to payments.csv
Saving productlines.csv to productlines.csv
Saving products.csv to products.csv


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkColabSetup") \
    .getOrCreate()

# Check version
print("✅ Spark Version:", spark.version)

✅ Spark Version: 3.3.2


In [4]:
# Example for loading and printing schema
products_df = spark.read.csv("/content/products.csv", header=True, inferSchema=True)
products_df.printSchema()
customers_df = spark.read.csv("/content/customers.csv",header=True,inferSchema=True)
customers_df.printSchema()
orders_df = spark.read.csv("/content/orders.csv",header=True,inferSchema=True)
orders_df.printSchema()
od_df = spark.read.csv("/content/orderdetails.csv",header=True,inferSchema=True)
od_df.printSchema()
offices_df = spark.read.csv("/content/offices.csv",header=True,inferSchema=True)
offices_df.printSchema()
employees_df = spark.read.csv("/content/employees.csv",header=True,inferSchema=True)
employees_df.printSchema()
payments_df = spark.read.csv("/content/payments.csv",header=True,inferSchema=True)
payments_df.printSchema()
pl_df = spark.read.csv("/content/productlines.csv",header=True,inferSchema=True)
pl_df.printSchema()

root
 |-- productCode: string (nullable = true)
 |-- productName: string (nullable = true)
 |-- productLine: string (nullable = true)
 |-- productScale: string (nullable = true)
 |-- productVendor: string (nullable = true)
 |-- productDescription: string (nullable = true)
 |-- quantityInStock: string (nullable = true)
 |-- buyPrice: string (nullable = true)
 |-- MSRP: string (nullable = true)

root
 |-- customerNumber: integer (nullable = true)
 |-- customerName: string (nullable = true)
 |-- contactLastName: string (nullable = true)
 |-- contactFirstName: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- addressLine1: string (nullable = true)
 |-- addressLine2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- country: string (nullable = true)
 |-- salesRepEmployeeNumber: double (nullable = true)
 |-- creditLimit: double (nullable = true)

root
 |-- orderNumber: integer (nul

In [5]:
joined_df=orders_df.join(od_df , orders_df['orderNumber']==od_df['orderNumber'])
joined_df.show()

+-----------+-------------------+-------------------+-------------------+-------+--------------------+--------------+-----------+-----------+---------------+---------+---------------+
|orderNumber|          orderDate|       requiredDate|        shippedDate| status|            comments|customerNumber|orderNumber|productCode|quantityOrdered|priceEach|orderLineNumber|
+-----------+-------------------+-------------------+-------------------+-------+--------------------+--------------+-----------+-----------+---------------+---------+---------------+
|      10100|2003-01-06 00:00:00|2003-01-13 00:00:00|2003-01-10 00:00:00|Shipped|                null|           363|      10100|   S18_1749|             30|    136.0|              3|
|      10100|2003-01-06 00:00:00|2003-01-13 00:00:00|2003-01-10 00:00:00|Shipped|                null|           363|      10100|   S18_2248|             50|    55.09|              2|
|      10100|2003-01-06 00:00:00|2003-01-13 00:00:00|2003-01-10 00:00:00|Shipped

##2. DataFrames and Tables Creation

In [6]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkDFExample") \
    .getOrCreate()


In [7]:
products_df.createOrReplaceTempView('products')
customers_df.createOrReplaceTempView('customers')
orders_df.createOrReplaceTempView('orders')

In [8]:
spark.sql("SELECT * FROM products").show(110)

+-----------+--------------------+----------------+------------+--------------------+--------------------+--------------------+--------------------+---------------+
|productCode|         productName|     productLine|productScale|       productVendor|  productDescription|     quantityInStock|            buyPrice|           MSRP|
+-----------+--------------------+----------------+------------+--------------------+--------------------+--------------------+--------------------+---------------+
|   S10_1678|1969 Harley David...|     Motorcycles|        1:10|     Min Lin Diecast|This replica feat...|                7933|               48.81|           95.7|
|   S10_1949|1952 Alpine Renau...|    Classic Cars|        1:10|Classic Metal Cre...|Turnable front wh...|                7305|               98.58|          214.3|
|   S10_2016|1996 Moto Guzzi 1...|     Motorcycles|        1:10|Highway 66 Mini C...|Official Moto Guz...|                6625|               68.99|         118.94|
|   S10_46

In [9]:
from pyspark.sql.functions import sum,col
products_df.filter(products_df.productLine=='Motorcycles').show()
customers_df.groupBy('city').sum('creditLimit').show()
customers_df.join(orders_df, customers_df.customerNumber == orders_df.customerNumber).show()

+-----------+--------------------+-----------+------------+--------------------+--------------------+---------------+--------+------+
|productCode|         productName|productLine|productScale|       productVendor|  productDescription|quantityInStock|buyPrice|  MSRP|
+-----------+--------------------+-----------+------------+--------------------+--------------------+---------------+--------+------+
|   S10_1678|1969 Harley David...|Motorcycles|        1:10|     Min Lin Diecast|This replica feat...|           7933|   48.81|  95.7|
|   S10_2016|1996 Moto Guzzi 1...|Motorcycles|        1:10|Highway 66 Mini C...|Official Moto Guz...|           6625|   68.99|118.94|
|   S10_4698|2003 Harley-David...|Motorcycles|        1:10|   Red Start Diecast|Model features, o...|           5582|   91.02|193.66|
|   S12_2823|    2002 Suzuki XREO|Motorcycles|        1:12|Unimax Art Galleries|Official logos an...|           9997|   66.27|150.62|
|   S18_2625|1936 Harley David...|Motorcycles|        1:18|Wel

## 3. Data Transformations and Aggregations

In [10]:
joined_df = od_df.join(orders_df,orders_df.orderNumber == od_df.orderNumber)
joined_df=joined_df.withColumn("Total",col('quantityOrdered')*col('priceEach'))
tot_cus=joined_df.groupBy('customerNumber').sum('Total')
tot_cus.show()

+--------------+------------------+
|customerNumber|        sum(Total)|
+--------------+------------------+
|           148|         156251.03|
|           496|         137460.79|
|           471|          55866.02|
|           458|112440.08999999998|
|           321|132340.77999999997|
|           362|          84340.32|
|           211| 45480.78999999999|
|           385| 87468.29999999999|
|           406| 86436.96999999999|
|           412| 90332.37999999996|
|           103|          22314.36|
|           350|          71547.53|
|           333|          55190.16|
|           128|          75937.76|
|           209|          75859.32|
|           319| 78432.16000000002|
|           233|          68977.67|
|           157|         104358.69|
|           177|62361.219999999994|
|           353|126983.18999999997|
+--------------+------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import min
from pyspark.sql.types import IntegerType

# Try converting 'quantityInStock' to Integer; invalid entries will become null
df = products_df.withColumn("quantityInStock", col("quantityInStock").cast(IntegerType()))

# Drop rows where conversion failed (non-numeric entries)
df_cleaned = df.filter(col("quantityInStock").isNotNull())

prod_low=df_cleaned.groupBy('productLine').min('quantityInStock')
prod_low.show()

+----------------+--------------------+
|     productLine|min(quantityInStock)|
+----------------+--------------------+
|     Motorcycles|                  15|
|    Vintage Cars|                 136|
|           Ships|                 414|
|Trucks and Buses|                 814|
|    Classic Cars|                  68|
|          Trains|                1645|
|          Planes|                 551|
+----------------+--------------------+



In [12]:
top_5=payments_df.groupBy('customerNumber').sum('amount').orderBy('sum(amount)',ascending=False)
top_5.show()

+--------------+------------------+
|customerNumber|       sum(amount)|
+--------------+------------------+
|           141| 715738.9800000001|
|           124| 584188.2400000001|
|           114|180585.06999999998|
|           151|         177913.95|
|           148|         156251.03|
|           323|154622.08000000002|
|           187|         148410.09|
|           276|         137034.22|
|           321|         132340.78|
|           146|         130305.35|
|           278|         127529.69|
|           353|         126983.19|
|           119|116949.68000000001|
|           363|116449.29000000001|
|           496|         114497.19|
|           458|         112440.09|
|           298|         108777.92|
|           131|         107639.94|
|           145|          107446.5|
|           398|         105548.73|
+--------------+------------------+
only showing top 20 rows



In [13]:
from pyspark.sql.functions import month, year, count
from pyspark.sql.window import Window

orders_df = orders_df.withColumn("orderMonth", month("orderDate")).withColumn("orderYear", year("orderDate"))
monthly_trend = orders_df.groupBy("orderYear", "orderMonth").agg(count("*").alias("orderCount"))
monthly_trend.orderBy("orderYear", "orderMonth").show()


+---------+----------+----------+
|orderYear|orderMonth|orderCount|
+---------+----------+----------+
|     2003|         1|         5|
|     2003|         2|         3|
|     2003|         3|         6|
|     2003|         4|         7|
|     2003|         5|         6|
|     2003|         6|         7|
|     2003|         7|         7|
|     2003|         8|         5|
|     2003|         9|         8|
|     2003|        10|        18|
|     2003|        11|        30|
|     2003|        12|         9|
|     2004|         1|         8|
|     2004|         2|        11|
|     2004|         3|         8|
|     2004|         4|        10|
|     2004|         5|         8|
|     2004|         6|        12|
|     2004|         7|        11|
|     2004|         8|        12|
+---------+----------+----------+
only showing top 20 rows



In [14]:
products_df.createOrReplaceTempView("products")
orders_df.createOrReplaceTempView("orders")
od_df.createOrReplaceTempView("orderdetails")
payments_df.createOrReplaceTempView("payments")
customers_df.createOrReplaceTempView("customers")

spark.sql("""
    SELECT c.customerName, SUM(od.quantityOrdered * od.priceEach) as totalOrder
    FROM customers c
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    GROUP BY c.customerName
    ORDER BY totalOrder DESC
""").show()


+--------------------+------------------+
|        customerName|        totalOrder|
+--------------------+------------------+
|Euro+ Shopping Ch...| 820689.5399999995|
|Mini Gifts Distri...|         591827.34|
|Australian Collec...|180585.06999999998|
|  Muscle Machine Inc|177913.95000000007|
|   La Rochelle Gifts|         158573.12|
|Dragon Souveniers...|         156251.03|
|Down Under Souven...|154622.08000000002|
|   Land of Toys Inc.|         149085.15|
|      AV Stores, Co.|148410.09000000003|
|The Sharp Gifts W...|         143536.27|
|Salzburg Collecta...|137480.07000000004|
|   Kelly's Gift Shop|137460.78999999995|
|Anna's Decoration...|         137034.22|
|Souveniers And Th...|133907.12000000002|
|Corporate Gift Id...|132340.77999999997|
|Saveley & Henriot...|130305.34999999998|
|Danish Wholesale ...|129085.12000000001|
|       Rovelli Gifts|         127529.69|
|  Reims Collectables|126983.18999999997|
| L'ordine Souveniers|125505.56999999999|
+--------------------+------------

##4. Data Export and GitHub Submission

In [15]:
top_5.coalesce(1).write.csv("output/top_customers", header=True,mode='overwrite')
monthly_trend.coalesce(1).write.csv("output/monthly_trends", header=True,mode='overwrite')
tot_cus.coalesce(1).write.csv("output/order_value_by_customer", header=True,mode='overwrite')
prod_low.coalesce(1).write.csv("output/lowest_stock_product.csv",header=True,mode='overwrite')

##5. Advanced SQL Queries

In [16]:
most_ordered=joined_df.groupBy('productCode').sum('quantityOrdered')
most_order=joined_df.groupBy('productCode').sum('Total')
most_ordered.join(most_order,on='productCode').show()


+-----------+--------------------+------------------+
|productCode|sum(quantityOrdered)|        sum(Total)|
+-----------+--------------------+------------------+
|   S18_4600|                1061|114232.79000000001|
|   S18_1749|                 918|140535.60000000003|
|   S18_2248|                 832|45306.770000000004|
|   S12_3891|                 965|         152543.02|
|  S700_1138|                 934|          56455.11|
|   S32_1268|                 911| 78067.81999999999|
|   S18_2795|                 880|132275.97999999998|
|   S12_1099|                 933|161531.47999999992|
|   S24_1937|                 937|          28052.94|
|   S32_3522|                 988|          57282.49|
|   S18_1097|                 999|105563.12000000001|
|   S18_1662|                1040|144959.90999999997|
|   S12_1666|                 972|119085.24999999999|
|   S24_3969|                 824|          29763.39|
|   S24_4048|                 867|           92973.4|
|   S24_1578|               

In [17]:
payments_df.show()

+--------------+-----------+-------------------+---------+
|customerNumber|checkNumber|        paymentDate|   amount|
+--------------+-----------+-------------------+---------+
|           103|   HQ336336|2004-10-19 00:00:00|  6066.78|
|           103|   JM555205|2003-06-05 00:00:00| 14571.44|
|           103|   OM314933|2004-12-18 00:00:00|  1676.14|
|           112|   BO864823|2004-12-17 00:00:00| 14191.12|
|           112|    HQ55022|2003-06-06 00:00:00| 32641.98|
|           112|   ND748579|2004-08-20 00:00:00| 33347.88|
|           114|    GG31455|2003-05-20 00:00:00| 45864.03|
|           114|   MA765515|2004-12-15 00:00:00| 82261.22|
|           114|   NP603840|2003-05-31 00:00:00|  7565.08|
|           114|    NR27552|2004-03-10 00:00:00| 44894.74|
|           119|   DB933704|2004-11-14 00:00:00| 19501.82|
|           119|   LN373447|2004-08-08 00:00:00| 47924.19|
|           119|    NG94694|2005-02-22 00:00:00| 49523.67|
|           121|   DB889831|2003-02-16 00:00:00| 50218.9

In [18]:
joined_df = pl_df.alias("pl") \
    .join(products_df.alias("p"), col("pl.productLine") == col("p.productLine")) \
    .join(od_df.alias("od"), col("p.productCode") == col("od.productCode")) \
    .join(orders_df.alias("o"), col("od.orderNumber") == col("o.orderNumber")) \
    .join(customers_df.alias("c"), col("o.customerNumber") == col("c.customerNumber")) \
    .select(
        col("pl.productLine"),
        col("p.productName"),
        col("od.quantityOrdered"),
        col("o.orderDate"),
        col("c.customerName")
    )
joined_df.show()

+-----------+--------------------+---------------+-------------------+--------------------+
|productLine|         productName|quantityOrdered|          orderDate|        customerName|
+-----------+--------------------+---------------+-------------------+--------------------+
|Motorcycles|1969 Harley David...|             66|2005-05-13 00:00:00|Euro+ Shopping Ch...|
|Motorcycles|1969 Harley David...|             24|2005-04-08 00:00:00|UK Collectables, ...|
|Motorcycles|1969 Harley David...|             40|2005-04-01 00:00:00|   Kelly's Gift Shop|
|Motorcycles|1969 Harley David...|             42|2005-03-03 00:00:00|    FunGiftIdeas.com|
|Motorcycles|1969 Harley David...|             21|2005-02-03 00:00:00|   La Rochelle Gifts|
|Motorcycles|1969 Harley David...|             20|2004-12-17 00:00:00|Souveniers And Th...|
|Motorcycles|1969 Harley David...|             42|2004-12-04 00:00:00|Down Under Souven...|
|Motorcycles|1969 Harley David...|             41|2004-11-24 00:00:00|Salzburg C

##6. Architecture and Documentation

In [ ]:
#. CSV Data → PySpark DataFrame → SQL Views → Aggregations & Joins → Export CSV → GitHub